## Train with imgaug generator

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
from skimage.transform import resize
import keras
import cv2
%matplotlib inline

import train_util
import util
from image_generator import Generator
%load_ext autoreload
%autoreload 2


/root/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Make {'image_id':depth} dictionary and train-val split

In [3]:
master_df = pd.read_csv('./data/depth_prop_cls_bad_mask.csv')
id_depth_df = master_df[['id', 'z']].set_index('id')['z']
depth_dict = id_depth_df.to_dict()

In [4]:
# data_type is 'train', 'test' and 'bad_train'
train_val_df = master_df[(master_df['data_type'] == 'train')]
ids = train_val_df['id'].values
classes = train_val_df['salt_propotion_class'].values

In [5]:
from sklearn.cross_validation import train_test_split
train_ids, val_ids = train_test_split(ids, test_size=0.2, random_state=43, stratify=classes)

In [14]:
# make generator
batch_size = 32
tgs_generator = Generator(train_ids = train_ids, depth_dict=depth_dict, val_ids=val_ids, batch_size=batch_size, feature_out=False)

In [15]:
# steps_per_epoch
steps_per_epoch = len(train_ids) //  batch_size
validation_steps = len(val_ids) //  batch_size
# validation batch will be all validation
# validation_steps = 1
print(steps_per_epoch)
print(validation_steps)

97
24


### Make val data

In [16]:
# X_val = np.zeros((len(val_ids), 128, 128, 3), dtype=np.float32)
# X_feat_val = np.zeros((len(val_ids), 1), dtype=np.float32)
# Y_val_true = np.zeros((len(val_ids), 128, 128, 1), dtype=np.float32)

# for i, im_id in enumerate(val_ids):
# #     image
#     im = cv2.imread('./data/train/images/' + im_id + '.png')
#     im = cv2.resize(im, (128, 128))
#     im = np.array(im, dtype='float32')
#     im /= 255
#     X_val[i] = im
#     #     feature
#     X_feat_val[i] = depth_dict[im_id]
# #     mask
#     mask = cv2.imread('./data/train/masks/' + im_id + '.png')
#     mask = cv2.resize(mask, (128, 128), interpolation=cv2.INTER_NEAREST)
#     mask = np.array((mask > 127.5), dtype='float32')
#     mask = np.expand_dims(mask[:, :, 0], axis=-1)
#     Y_val_true[i] = mask

# val_gen = ([X_val, X_feat_val], Y_val_true)

### Model

In [21]:
# unet_m = util.unet_model(min_filter_num=16, with_vec=True)

# lighter and better!!
img_size_target = 128
unet_m = util.UNet((img_size_target,img_size_target,3),start_ch=16,depth=5,batchnorm=True)

In [22]:
# unet_m.summary()

## Train

In [ ]:
# optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=True) 
# Todo Adam 
# train_util.train(unet_m, tgs_generator.generate(True), tgs_generator.generate(False), loss=train_util.dice_p_bce, metrics=[train_util.dice_coef, 'binary_accuracy', train_util.true_positive_rate, train_util.iou_coef], epochs=3000, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)
train_util.train(unet_m, 
                 tgs_generator.generate(True), 
                 tgs_generator.generate(False), 
                 metrics=[train1_util.dice_coef, 'binary_accuracy', train_util.true_positive_rate, train_util.iou_coef, train_util.mean_iou_threshold],
                 epochs=3000, steps_per_epoch=steps_per_epoch, 
                 validation_steps=validation_steps,
                loss=train_util.dice_p_bce)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_69 (Conv2D)              (None, 128, 128, 16) 448         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_57 (BatchNo (None, 128, 128, 16) 64          conv2d_69[0][0]                  
__________________________________________________________________________________________________
conv2d_70 (Conv2D)              (None, 128, 128, 16) 2320        batch_normalization_57[0][0]     
__________________________________________________________________________________________________
batch_norm

Epoch 1/3000
 - 70s - loss: 1.2050 - dice_coef: 0.2836 - binary_accuracy: 0.8054 - true_positive_rate: 0.7175 - iou_coef: 0.2596 - mean_iou_threshold: 0.1693 - val_loss: 2.5355 - val_dice_coef: 0.1841 - val_binary_accuracy: 0.7964 - val_true_positive_rate: 0.3256 - val_iou_coef: 0.1423 - val_mean_iou_threshold: 0.0682
Finished epoch: 0
{'mean_iou_threshold': 0.169265464063465, 'iou_coef': 0.2596477313140004, 'val_true_positive_rate': 0.3255509802450736, 'val_binary_accuracy': 0.7964476744333903, 'val_dice_coef': 0.18405898877729973, 'loss': 1.2049593089782085, 'val_iou_coef': 0.14228299741322795, 'true_positive_rate': 0.7174504279475851, 'val_loss': 2.5355235785245895, 'binary_accuracy': 0.8054291440039566, 'dice_coef': 0.28364324369995864, 'val_mean_iou_threshold': 0.06822916663562258}

Epoch 00001: val_loss improved from inf to 2.53552, saving model to ./log/2018_0811_1836/best_weights.hdf5
Epoch 2/3000
 - 57s - loss: 1.0059 - dice_coef: 0.3178 - binary_accuracy: 0.8876 - true_positi

Finished epoch: 9
{'mean_iou_threshold': 0.5200708769645888, 'iou_coef': 0.5787707296843382, 'val_true_positive_rate': 0.8157027016083399, 'val_binary_accuracy': 0.9251734415690104, 'val_dice_coef': 0.40494970604777336, 'loss': 0.8279928687921504, 'val_iou_coef': 0.642653060456117, 'true_positive_rate': 0.8515882891477998, 'val_loss': 0.7994668955604235, 'binary_accuracy': 0.919982870829474, 'lr': 0.001, 'dice_coef': 0.3861907330370441, 'val_mean_iou_threshold': 0.5796875034769376}

Epoch 00010: val_loss did not improve
Epoch 11/3000
 - 61s - loss: 0.8161 - dice_coef: 0.3937 - binary_accuracy: 0.9210 - true_positive_rate: 0.8551 - iou_coef: 0.5925 - mean_iou_threshold: 0.5384 - val_loss: 0.7190 - val_dice_coef: 0.4391 - val_binary_accuracy: 0.9409 - val_true_positive_rate: 0.8895 - val_iou_coef: 0.7144 - val_mean_iou_threshold: 0.6634
Finished epoch: 10
{'mean_iou_threshold': 0.5384020633304242, 'iou_coef': 0.592482211663551, 'val_true_positive_rate': 0.8894869337479273, 'val_binary_ac

 - 60s - loss: 0.7315 - dice_coef: 0.5769 - binary_accuracy: 0.9019 - true_positive_rate: 0.7879 - iou_coef: 0.6147 - mean_iou_threshold: 0.5564 - val_loss: 3.6379 - val_dice_coef: 0.4165 - val_binary_accuracy: 0.7659 - val_true_positive_rate: 0.0160 - val_iou_coef: 0.4121 - val_mean_iou_threshold: 0.3999
Finished epoch: 19
{'mean_iou_threshold': 0.5564110816754017, 'iou_coef': 0.6146882840038574, 'val_true_positive_rate': 0.01603035646257922, 'val_binary_accuracy': 0.7659447193145752, 'val_dice_coef': 0.41654359300931293, 'loss': 0.7314810436411002, 'val_iou_coef': 0.4120517522096634, 'true_positive_rate': 0.787887565868417, 'val_loss': 3.6379289031028748, 'binary_accuracy': 0.9019067901925942, 'lr': 0.001, 'dice_coef': 0.5768828923554764, 'val_mean_iou_threshold': 0.39986979340513545}

Epoch 00020: val_loss did not improve
Epoch 21/3000
 - 61s - loss: 0.7699 - dice_coef: 0.5692 - binary_accuracy: 0.8915 - true_positive_rate: 0.7754 - iou_coef: 0.6183 - mean_iou_threshold: 0.5628 - va

Finished epoch: 28
{'mean_iou_threshold': 0.6326675285998079, 'iou_coef': 0.678378583844175, 'val_true_positive_rate': 0.8645255838831266, 'val_binary_accuracy': 0.9507347742716471, 'val_dice_coef': 0.7374316478768984, 'loss': 0.5634953447223938, 'val_iou_coef': 0.7401718273758888, 'true_positive_rate': 0.8500501052620485, 'val_loss': 0.46389597095549107, 'binary_accuracy': 0.924648520872765, 'lr': 0.00020000001, 'dice_coef': 0.661104881579114, 'val_mean_iou_threshold': 0.6967447996139526}

Epoch 00029: val_loss improved from 0.50618 to 0.46390, saving model to ./log/2018_0811_1836/best_weights.hdf5
Epoch 30/3000
 - 62s - loss: 0.5885 - dice_coef: 0.6531 - binary_accuracy: 0.9247 - true_positive_rate: 0.8430 - iou_coef: 0.6804 - mean_iou_threshold: 0.6307 - val_loss: 0.4815 - val_dice_coef: 0.7061 - val_binary_accuracy: 0.9468 - val_true_positive_rate: 0.8588 - val_iou_coef: 0.7354 - val_mean_iou_threshold: 0.6897
Finished epoch: 29
{'mean_iou_threshold': 0.6307345346077201, 'iou_coef'

...

 - 62s - loss: 0.4467 - dice_coef: 0.7406 - binary_accuracy: 0.9430 - true_positive_rate: 0.8801 - iou_coef: 0.7526 - mean_iou_threshold: 0.7118 - val_loss: 0.3770 - val_dice_coef: 0.8017 - val_binary_accuracy: 0.9570 - val_true_positive_rate: 0.8842 - val_iou_coef: 0.7977 - val_mean_iou_threshold: 0.7635
Finished epoch: 727
{'mean_iou_threshold': 0.7117912357615441, 'iou_coef': 0.7525731146950083, 'val_true_positive_rate': 0.8841553976138433, 'val_binary_accuracy': 0.9570410251617432, 'val_dice_coef': 0.801730751991272, 'loss': 0.4466856622818819, 'val_iou_coef': 0.7977245499690374, 'true_positive_rate': 0.8800590812545461, 'val_loss': 0.3770162748793761, 'binary_accuracy': 0.9430202602111187, 'lr': 0.0, 'dice_coef': 0.7406458940702615, 'val_mean_iou_threshold': 0.7635416636864344}

Epoch 00728: val_loss did not improve
Epoch 729/3000
 - 64s - loss: 0.4339 - dice_coef: 0.7430 - binary_accuracy: 0.9441 - true_positive_rate: 0.8870 - iou_coef: 0.7560 - mean_iou_threshold: 0.7168 - val_l

 - 63s - loss: 0.4348 - dice_coef: 0.7419 - binary_accuracy: 0.9453 - true_positive_rate: 0.8918 - iou_coef: 0.7552 - mean_iou_threshold: 0.7135 - val_loss: 0.3804 - val_dice_coef: 0.8023 - val_binary_accuracy: 0.9571 - val_true_positive_rate: 0.8842 - val_iou_coef: 0.7978 - val_mean_iou_threshold: 0.7637
Finished epoch: 737
{'mean_iou_threshold': 0.7135309282037401, 'iou_coef': 0.7552245044216668, 'val_true_positive_rate': 0.8841891462604204, 'val_binary_accuracy': 0.95705246925354, 'val_dice_coef': 0.8022708917657534, 'loss': 0.4348222043403645, 'val_iou_coef': 0.797777292629083, 'true_positive_rate': 0.891786339971208, 'val_loss': 0.38038821953038376, 'binary_accuracy': 0.9452743333639558, 'lr': 0.0, 'dice_coef': 0.7418849892223004, 'val_mean_iou_threshold': 0.7636718700329462}

Epoch 00738: val_loss did not improve
Epoch 739/3000
 - 58s - loss: 0.4449 - dice_coef: 0.7341 - binary_accuracy: 0.9436 - true_positive_rate: 0.8859 - iou_coef: 0.7493 - mean_iou_threshold: 0.7101 - val_los

 - 61s - loss: 0.4534 - dice_coef: 0.7379 - binary_accuracy: 0.9466 - true_positive_rate: 0.8901 - iou_coef: 0.7517 - mean_iou_threshold: 0.7147 - val_loss: 0.3819 - val_dice_coef: 0.8018 - val_binary_accuracy: 0.9571 - val_true_positive_rate: 0.8839 - val_iou_coef: 0.8002 - val_mean_iou_threshold: 0.7656
Finished epoch: 747
{'mean_iou_threshold': 0.7147229381443299, 'iou_coef': 0.7516774178780231, 'val_true_positive_rate': 0.8839219907919565, 'val_binary_accuracy': 0.9571399688720703, 'val_dice_coef': 0.8017504339416822, 'loss': 0.45343635008507166, 'val_iou_coef': 0.8001749242345492, 'true_positive_rate': 0.8900564736926678, 'val_loss': 0.3819459894051154, 'binary_accuracy': 0.9465557766943863, 'lr': 0.0, 'dice_coef': 0.737931514523693, 'val_mean_iou_threshold': 0.7656249950329462}

Epoch 00748: val_loss did not improve
Epoch 749/3000
 - 61s - loss: 0.4577 - dice_coef: 0.7375 - binary_accuracy: 0.9437 - true_positive_rate: 0.8925 - iou_coef: 0.7527 - mean_iou_threshold: 0.7128 - val_

 - 63s - loss: 0.4441 - dice_coef: 0.7448 - binary_accuracy: 0.9440 - true_positive_rate: 0.8895 - iou_coef: 0.7604 - mean_iou_threshold: 0.7196 - val_loss: 0.3770 - val_dice_coef: 0.8013 - val_binary_accuracy: 0.9572 - val_true_positive_rate: 0.8852 - val_iou_coef: 0.7968 - val_mean_iou_threshold: 0.7626
Finished epoch: 757
{'mean_iou_threshold': 0.7195876271454329, 'iou_coef': 0.7604009367756008, 'val_true_positive_rate': 0.8851858451962471, 'val_binary_accuracy': 0.9572270711263021, 'val_dice_coef': 0.8012860218683878, 'loss': 0.44409068097773285, 'val_iou_coef': 0.7967827171087265, 'true_positive_rate': 0.8895196601287606, 'val_loss': 0.3769718116770188, 'binary_accuracy': 0.9440240565034532, 'lr': 0.0, 'dice_coef': 0.7448420874851266, 'val_mean_iou_threshold': 0.7626302043596903}

Epoch 00758: val_loss did not improve
Epoch 759/3000
 - 63s - loss: 0.4412 - dice_coef: 0.7431 - binary_accuracy: 0.9459 - true_positive_rate: 0.8871 - iou_coef: 0.7556 - mean_iou_threshold: 0.7158 - val

 - 62s - loss: 0.4617 - dice_coef: 0.7330 - binary_accuracy: 0.9437 - true_positive_rate: 0.8890 - iou_coef: 0.7422 - mean_iou_threshold: 0.7029 - val_loss: 0.3783 - val_dice_coef: 0.8016 - val_binary_accuracy: 0.9572 - val_true_positive_rate: 0.8841 - val_iou_coef: 0.7966 - val_mean_iou_threshold: 0.7626
Finished epoch: 767
{'mean_iou_threshold': 0.7029317006007912, 'iou_coef': 0.7421752060811544, 'val_true_positive_rate': 0.884076808889707, 'val_binary_accuracy': 0.9572126865386963, 'val_dice_coef': 0.8016113440195719, 'loss': 0.4617022993331103, 'val_iou_coef': 0.7965866054097811, 'true_positive_rate': 0.8890191746741226, 'val_loss': 0.37830199177066487, 'binary_accuracy': 0.9436508453998369, 'lr': 0.0, 'dice_coef': 0.7330011639398398, 'val_mean_iou_threshold': 0.7626302043596903}

Epoch 00768: val_loss did not improve
Epoch 769/3000
 - 58s - loss: 0.4464 - dice_coef: 0.7408 - binary_accuracy: 0.9427 - true_positive_rate: 0.8900 - iou_coef: 0.7520 - mean_iou_threshold: 0.7124 - val_

## Check iou_coef

In [7]:
from keras.models import load_model
model = load_model('./log/2018_0807_2002/best_weights.hdf5', custom_objects={
    'mean_iou': train_util.mean_iou, 
    'dice_p_bce':train_util.dice_p_bce, 
    'dice_coef':train_util.dice_coef,
    'true_positive_rate':train_util.true_positive_rate,
    'iou_coef':train_util.iou_coef
})

In [29]:
preds = model.predict([X_val, X_feat_val])

In [30]:
preds_t = (preds > 0.5).astype(np.uint8)

In [33]:
ious = []
for i in range(0, len(preds_t)):
    y_true = Y_val_true[i]
    y_pred = preds_t[i]
    ious.append(train_util.compute_iou(y_true, y_pred))

In [36]:
np.mean(ious)

0.7901850010723667

In [37]:
len(train_util.BAD_MASKS)

113

In [41]:
113/(len(preds) * 5)

0.02904884318766067

In [45]:
0.79/1.02

0.7745098039215687

In [52]:
iou = train_util.iou_coef(Y_val_true, np.float32(preds_t))

In [58]:
from keras import backend as K
K.get_value(K.mean((iou)))

0.78863376

In [ ]:
train